In [1]:
import os
import glob
import tensorflow as tf
from absl import logging
from modules import components, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

c:\Users\zis\anaconda3\envs\movies-rec\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
PIPELINE_NAME = "movie_recommender_pipeline"

# pipeline inputs
DATA_ROOT = "data/ratings"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"

# pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

In [3]:
components = components.init_components(
    data_dir=DATA_ROOT,
    transform_module=TRANSFORM_MODULE_FILE,
    tuner_module=TUNER_MODULE_FILE,
    trainer_module=TRAINER_MODULE_FILE,
    train_steps=500,
    eval_steps=100,
    serving_model_dir=os.path.join(
        serving_model_dir, "cf_model"),
    epochs=5,
)

pipeline = pipeline.init_pipeline(
    pipeline_root=pipeline_root,
    pipeline_name=PIPELINE_NAME,
    metadata_path=metadata_path,
    components=components,
)

BeamDagRunner().run(pipeline)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: outputs\movie_recommender_pipeline\Transform\transform_graph\19\.temp_path\tftransform_tmp\22ab99d6b24e447eae57b3b9717685b8\assets


INFO:tensorflow:Assets written to: outputs\movie_recommender_pipeline\Transform\transform_graph\19\.temp_path\tftransform_tmp\22ab99d6b24e447eae57b3b9717685b8\assets


INFO:tensorflow:struct2tensor is not available.


...

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 userid_xf (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 movieid_xf (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 16)        10704       ['userid_xf[0][0]']              
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 16)        152304      ['movieid_xf[0][0]']             
                                                                                              

ERROR:absl:ERROR IN _get_cf_model:
Invalid keyword argument(s) in `compile()`: ({'optimizers'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".


Search space summary
Default search space size: 6
embedding_dims (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 1024, 'step': 32, 'sampling': None}
l2_regularizers (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001, 1e-05], 'ordered': True}
num_hidden_layers (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
dense_unit (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 512, 'step': 32, 'sampling': None}
dropout_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.8, 'step': 0.1, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001, 1e-05], 'ordered': True}

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
688               |?                 |embedding_dims
0.0001            |?                 |l2_regularizers
2                 |?                 |num_hidden_layers
200         

ERROR:absl:ERROR IN _get_cf_model:
Invalid keyword argument(s) in `compile()`: ({'optimizers'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".
ERROR:absl:Execution 21 failed.


RuntimeError: Model-building function did not return a valid Keras Model instance, found None [while running 'Run[Tuner]']